In [1]:
import os
import gc
import numpy as np
import pandas as pd
import joblib
from datetime import datetime

from lightgbm import LGBMClassifier
import optuna
from prunedcv import PrunedCV

from codes.utils import import_data, drop_columns, cross_val_score_auc, reduce_mem_usage
from codes.fe_browser import latest
from codes.fe_emails import proton, mappings, labeling
from codes.fe_cards import stats
from codes.fe_date import dates
from codes.fe_relatives import divisions
from codes.fe_categorical import pairs, wtf
from codes.prepro import prepro
from codes.fe_users import users_stats

from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin

/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
DATA_PATH = '../input/'
SEARCH_PARAMS = True
SEARCH_FEATURES = False
N_FOLD = 8

In [4]:
train, test, sample_submission = import_data(DATA_PATH)

### Some Feature Engineering

drop columns, count encoding, aggregation, fillna

In [5]:
train, test = users_stats(train, test)

# train, test = drop_columns(train, test)

train, test = latest(train, test)

train, test = proton(train, test)

train['nulls1'] = train.isna().sum(axis=1)
test['nulls1'] = test.isna().sum(axis=1)

train, test = mappings(train, test)
train, test = labeling(train, test)

train, test = stats(train, test)

train, test = divisions(train, test)

train, test = dates(train, test)

train, test = pairs(train, test)
train, test = wtf(train, test)

y_train = train['isFraud'].copy()


X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

del train, test

#fill in mean for floats
X_train, X_test = prepro(X_train, X_test)

/Users/piotrgabrys/data/python3/ieee/codes/fe_users.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_data = pd.concat([train, test])


### Model and training

In [6]:
columns = list(set(
['C{}'.format(i) for i in range(1,15)] \
+ ['D{}'.format(i) for i in range(1,16)] \
+ ['V' + str(i) for i in range(1,340)]))

for col in columns:
    if col in X_train.columns:
        X_train[col + '_' + 'trx'] = X_train[col] / X_train.TransactionAmt
        X_test[col + '_' + 'trx'] = X_test[col] / X_test.TransactionAmt

In [7]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

Mem. usage decreased to 1944.96 Mb (6.7% reduction)
Mem. usage decreased to 1653.57 Mb (6.7% reduction)


In [8]:
X_train[X_train == np.inf] = -1
X_train[X_train == -np.inf] = -1
X_test[X_test == np.inf] = -1
X_test[X_test == -np.inf] = -1
X_train[X_test.isna()] = -1
X_test[X_test.isna()] = -1

In [9]:
X_test.drop(['TransactionDT'], axis=1, inplace=True)
X_train.drop(['TransactionDT'], axis=1, inplace=True)

In [37]:
sfm = SelectFromModel(LGBMClassifier(metric='auc'), threshold=0.5)
sfm.fit(X_train, y_train)


print(X_train.shape[1])
columns = list(X_train.columns[sfm.get_support()])
print(len(columns))
X_train = X_train.loc[:,columns]
X_test = X_test.loc[:,columns]

SelectFromModel(estimator=LGBMClassifier(boosting_type='gbdt',
                                         class_weight=None,
                                         colsample_bytree=1.0,
                                         importance_type='split',
                                         learning_rate=0.1, max_depth=-1,
                                         metric='auc', min_child_samples=20,
                                         min_child_weight=0.001,
                                         min_split_gain=0.0, n_estimators=100,
                                         n_jobs=-1, num_leaves=31,
                                         objective=None, random_state=None,
                                         reg_alpha=0.0, reg_lambda=0.0,
                                         silent=True, subsample=1.0,
                                         subsample_for_bin=200000,
                                         subsample_freq=0),
                max_features=None, norm_or

In [46]:
class Counter(TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print(X.shape[1])
        return X

In [56]:
model = make_pipeline(
    SelectFromModel(LGBMClassifier(metric='auc')),
#     Counter(),
    LGBMClassifier(metric='auc',
                   n_estimators=1000)
)

In [48]:
prun = PrunedCV(N_FOLD, 0.03, minimize=False)

In [50]:
def objective(trial):
    
    joblib.dump(study, 'study.pkl') 

    
    params = {
        'selectfrommodel__threshold': trial.suggest_int('selectfrommodel__threshold', 1, 100),
        'lgbmclassifier__num_leaves': trial.suggest_int('lgbmclassifier__num_leaves', 10, 1500), 
        'lgbmclassifier__subsample_for_bin': trial.suggest_int('lgbmclassifier__subsample_for_bin', 10, 3000000), 
        'lgbmclassifier__min_child_samples': trial.suggest_int('lgbmclassifier__min_child_samples', 2, 100000), 
        'lgbmclassifier__reg_alpha': trial.suggest_loguniform('lgbmclassifier__reg_alpha', 0.00000000001, 10.0),
        'lgbmclassifier__colsample_bytree': trial.suggest_loguniform('lgbmclassifier__colsample_bytree', 0.0001, 1.0),
        'lgbmclassifier__learning_rate': trial.suggest_loguniform('lgbmclassifier__learning_rate', 0.000001, 10.0)
    }
    
    print(params)
    
    model.set_params(**params)
    return prun.cross_val_score(model, 
                                X_train, 
                                y_train, 
                                metric='auc', 
                                shuffle=True, 
                                random_state=42)

In [55]:
if SEARCH_PARAMS:
    if os.path.isfile('study.pkl'):
        study = joblib.load('study.pkl')
    else:
        study = optuna.create_study()

    study.optimize(objective, timeout=60 * 60 * 22)
    joblib.dump(study, 'study.pkl')
    best_params = study.best_params

else:

    best_params = {
        'selectfrommodel__threshold': 11,
        'lgbmclassifier__num_leaves': 330,
        'lgbmclassifier__subsample_for_bin': 2077193,
        'lgbmclassifier__min_child_samples': 2227,
        'lgbmclassifier__reg_alpha': 0.16758905622425835,
        'lgbmclassifier__colsample_bytree': 0.49030006727392056,
        'lgbmclassifier__learning_rate': 0.07916040470631734
    }

{'selectfrommodel__threshold': 11, 'lgbmclassifier__num_leaves': 290, 'lgbmclassifier__subsample_for_bin': 2022797, 'lgbmclassifier__min_child_samples': 8958, 'lgbmclassifier__reg_alpha': 0.0592854713909757, 'lgbmclassifier__colsample_bytree': 0.17295118760469255, 'lgbmclassifier__learning_rate': 0.10293221244710013}
74
74
66
66
73
73
80
80
73
73
79
79
73
73
72
72


[I 2019-09-11 09:10:17,136] Finished trial#125 resulted in value: -0.9567001742786652. Current best value is -0.9737159592210338 with parameters: {'selectfrommodel__threshold': 11, 'lgbmclassifier__num_leaves': 330, 'lgbmclassifier__subsample_for_bin': 2077193, 'lgbmclassifier__min_child_samples': 2227, 'lgbmclassifier__reg_alpha': 0.16758905622425835, 'lgbmclassifier__colsample_bytree': 0.49030006727392056, 'lgbmclassifier__learning_rate': 0.07916040470631734}.


{'selectfrommodel__threshold': 17, 'lgbmclassifier__num_leaves': 317, 'lgbmclassifier__subsample_for_bin': 1898697, 'lgbmclassifier__min_child_samples': 5589, 'lgbmclassifier__reg_alpha': 0.2601135670697154, 'lgbmclassifier__colsample_bytree': 0.3706504276055055, 'lgbmclassifier__learning_rate': 0.026075703002899617}
42
42
41
41
44
44
42
42
44
44
44
44
44
44
42
42


[I 2019-09-11 09:25:29,837] Finished trial#126 resulted in value: -0.9512384838876272. Current best value is -0.9737159592210338 with parameters: {'selectfrommodel__threshold': 11, 'lgbmclassifier__num_leaves': 330, 'lgbmclassifier__subsample_for_bin': 2077193, 'lgbmclassifier__min_child_samples': 2227, 'lgbmclassifier__reg_alpha': 0.16758905622425835, 'lgbmclassifier__colsample_bytree': 0.49030006727392056, 'lgbmclassifier__learning_rate': 0.07916040470631734}.


{'selectfrommodel__threshold': 6, 'lgbmclassifier__num_leaves': 513, 'lgbmclassifier__subsample_for_bin': 1722963, 'lgbmclassifier__min_child_samples': 2325, 'lgbmclassifier__reg_alpha': 0.08880634208100523, 'lgbmclassifier__colsample_bytree': 0.5170543055441995, 'lgbmclassifier__learning_rate': 0.1719791491681687}
143
143
151
151
144
144
144
144
150
150
139
139
144
144
148
148


...

[I 2019-09-12 06:45:07,585] Finished trial#167 resulted in value: -0.9734610021767773. Current best value is -0.9737159592210338 with parameters: {'selectfrommodel__threshold': 11, 'lgbmclassifier__num_leaves': 330, 'lgbmclassifier__subsample_for_bin': 2077193, 'lgbmclassifier__min_child_samples': 2227, 'lgbmclassifier__reg_alpha': 0.16758905622425835, 'lgbmclassifier__colsample_bytree': 0.49030006727392056, 'lgbmclassifier__learning_rate': 0.07916040470631734}.


{'selectfrommodel__threshold': 8, 'lgbmclassifier__num_leaves': 958, 'lgbmclassifier__subsample_for_bin': 1718383, 'lgbmclassifier__min_child_samples': 4655, 'lgbmclassifier__reg_alpha': 0.21540785375019736, 'lgbmclassifier__colsample_bytree': 0.31807977316922204, 'lgbmclassifier__learning_rate': 8.145501879041405e-06}
105
105
108
108


[I 2019-09-12 06:50:13,560] Finished trial#168 resulted in value: -0.8741745503381328. Current best value is -0.9737159592210338 with parameters: {'selectfrommodel__threshold': 11, 'lgbmclassifier__num_leaves': 330, 'lgbmclassifier__subsample_for_bin': 2077193, 'lgbmclassifier__min_child_samples': 2227, 'lgbmclassifier__reg_alpha': 0.16758905622425835, 'lgbmclassifier__colsample_bytree': 0.49030006727392056, 'lgbmclassifier__learning_rate': 0.07916040470631734}.


{'selectfrommodel__threshold': 15, 'lgbmclassifier__num_leaves': 939, 'lgbmclassifier__subsample_for_bin': 1911999, 'lgbmclassifier__min_child_samples': 72, 'lgbmclassifier__reg_alpha': 0.11805833543438835, 'lgbmclassifier__colsample_bytree': 0.09538171371383206, 'lgbmclassifier__learning_rate': 0.10893461260466755}
52
52
47
47
47
47
48
48
52
52
50
50
46
46
43
43


[I 2019-09-12 07:09:38,528] Finished trial#169 resulted in value: -0.973367222089144. Current best value is -0.9737159592210338 with parameters: {'selectfrommodel__threshold': 11, 'lgbmclassifier__num_leaves': 330, 'lgbmclassifier__subsample_for_bin': 2077193, 'lgbmclassifier__min_child_samples': 2227, 'lgbmclassifier__reg_alpha': 0.16758905622425835, 'lgbmclassifier__colsample_bytree': 0.49030006727392056, 'lgbmclassifier__learning_rate': 0.07916040470631734}.


In [57]:
model.set_params(**best_params)

cross_val_score_auc(model,
                    X_train,
                    y_train,
                    n_fold=N_FOLD,
                    stratify=True,
                    shuffle=True,
                    random_state=42,
                    predict=True,
                    X_test=X_test,
                    submission=sample_submission)

ROC accuracy: 0.9707565062294428, Train: 0.9999416292415686
ROC accuracy: 0.9758652343514882, Train: 0.9998818960438143
ROC accuracy: 0.9747893539459415, Train: 0.9999033992474002
ROC accuracy: 0.9741729952670382, Train: 0.999944229888998
ROC accuracy: 0.9735064735460197, Train: 0.9999515715657177
ROC accuracy: 0.9728703535857148, Train: 0.9999501665218518
ROC accuracy: 0.9746020273044912, Train: 0.9999374155994768
ROC accuracy: 0.973164729538134, Train: 0.9999402925194638




0.9737159592210338

In [ ]:
# ROC accuracy: 0.9707565062294428, Train: 0.9999416292415686
# ROC accuracy: 0.9758652343514882, Train: 0.9998818960438143
# ROC accuracy: 0.9747893539459415, Train: 0.9999033992474002
# ROC accuracy: 0.9741729952670382, Train: 0.999944229888998
# ROC accuracy: 0.9735064735460197, Train: 0.9999515715657177
# ROC accuracy: 0.9728703535857148, Train: 0.9999501665218518
# ROC accuracy: 0.9746020273044912, Train: 0.9999374155994768
# ROC accuracy: 0.973164729538134, Train: 0.9999402925194638


# 0.9737159592210338